In [1]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

In [2]:
def kmeans(x, k, Niter):
    initloc = np.random.choice(len(x), k, replace=False)
    centroids = x[initloc, :]
    
    dist = cdist(x,centroids, 'euclidean')
    points = np.array([np.argmin(i) for i in dist])
    
    for i in range(Niter):
        for initloc in range(k):
            temp_cent = x[points==initloc].mean(axis=0)
            centroids.append(temp_cent)
    
    centroids = np.vstack(centroids)
    distance = cdist(x,centroids, 'euclidean')
    points = np.array([np.argmin(i) for i in distance])
    
    return points
        

In [3]:
# read-in the excel spreadsheet using pandas 
df = pd.read_excel('Online_retail.xlsx', sheet_name='Online Retail')

# Remove  cancelled orders
df = df.loc[df['Quantity'] > 0]

# Remove any entries without CustomerID
df = df[pd.notnull(df['CustomerID'])]

# Remove incomplete month
df = df.loc[df['InvoiceDate'] < '2011-12-01']

# Calculate total sales from the Quantity and UnitPrice
df['Sales'] = df['Quantity'] * df['UnitPrice']

# use groupby to aggregate sales by CustomerID
customer_df = df.groupby('CustomerID').agg({'Sales': sum, 
                                            'InvoiceNo': lambda x: x.nunique()})

# Select the columns of interest
customer_df.columns = ['TotalSales', 'OrderCount'] 

# create a new column 'AvgOrderValue'
customer_df['AvgOrderValue'] = customer_df['TotalSales'] / customer_df['OrderCount']


rank_df = customer_df.rank(method='first')
# normalized_df = (rank_df - rank_df.mean()) / rank_df.std()
normalized_df =  2* (rank_df - rank_df.mean()) / (rank_df.max() - rank_df.min())
# normalized_df.head(10)

In [6]:
normalized_df['OrderCount']

CustomerID
12346.0   -1.000000
12347.0    0.614615
12348.0    0.331161
12349.0   -0.999535
12350.0   -0.999069
             ...   
18280.0   -0.285083
18281.0   -0.284617
18282.0   -0.284152
18283.0    0.930184
18287.0    0.330696
Name: OrderCount, Length: 4298, dtype: float64

In [7]:

label = kmeans([normalized_df['OrderCount'],normalized_df['TotalSales']], 4, 10)

#u_labels = np.unique(label)
#for i in u_labels:
#    plt.scatter(df[label== i, 0], df[label== i, 1], label = i)
#plt.legend()
#plt.grid()
#plt.show()

ValueError: Cannot take a larger sample than population when 'replace=False'